In [ ]:
import findspark
findspark.init()

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'


from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from operator import add


n_secs = 30
topic = "SparkKafkaProject"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[*]")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)

# After running zookeeper and kafka, lets start a stream of the Data in terminal.

kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'localhost:9092', 
                        'group.id':'video-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
    
lines = kafkaStream.map(lambda x: x[1])    # returns elements of type 155.156.35.110 - - [25/May/2015:23:11:15 +0000] "GET / HTTP/1.0" 200 3557 "-" "Mozilla/3.01 (compatible;)"
ip_pairs = lines.map(lambda x:(str(x.split('-')[0].strip()),1))   # ('ip', 1)
ip_counts = ip_pairs.reduceByKey(add)      # Adds up the elements of the above RDD if the ips are the same.
potential_attackers = ip_counts.filter(lambda x:x[1]>1).map(lambda x:x[0]) # returns the ips that have repeated more than once in a 30 second batch.
potential_attackers.pprint()


ssc.start()